### Simple POP model works

In [1]:
include("../LiPoSID.jl")
#using QuantumOptics
#basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5
#using TSSOS

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma\left[J \rho J^\dagger - \frac{1}{2}\left\{ J^\dagger J, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J = \begin{pmatrix} 0 & a \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma} \\ 0 & 0 
   \end{pmatrix}
$


In [2]:
@polyvar w
@polyvar a


Hˢʸᵐᵇ = [ w   0
          0   0. ]

2×2 Matrix{Term{Float64, Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}}:
 w    0.0
 0.0  0.0

In [3]:
method = "ansatz"

Jˢʸᵐᵇ = [ 0   a
          0   0 ]


2×2 Matrix{Term{Int64, Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}}:
 0  a
 0  0

In [4]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

../DATA/


In [5]:
all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;

In [6]:
function g_objective(γᵢ)

    objₑₓ = 0

    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [Jˢʸᵐᵇ])
        
    end # of files (initial states) loop

    return(objₑₓ)
end


g_objective (generic function with 1 method)

In [7]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
objectives_file = "POP_objectives_simple_" * date_and_time_string * ".h5"

models_dir = "../MODELS/"

γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in  γ_list
    pop_lme_obj = g_objective(γᵢ)

    println(γᵢ)

    h5open(models_dir*objectives_file,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)
        γ_group["pop_lme_objective"] = string(pop_lme_obj)
                        
    end 

end

println(models_dir*objectives_file)

0.079477


0.25133


0.79477


2.5133


7.9477


25.133


79.477


251.33
../MODELS/POP_objectives_simple_2024-Jan-13_at_11-03.h5
